In [57]:
# imoport libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [58]:
# import the data set of movie ratings 
df = pd.read_csv('../data/ml-latest-small/ratings.csv', na_values = 'Nan')

In [59]:
df.columns

Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')

In [60]:
# use pivot to make the matrix of movie rates
rates =df.pivot(index='userId',columns = 'movieId')
rates.head()

rating                                                                 \
movieId 1      2      3      4      5      6      7      8      9      10       
userId                                                                          
1          4.0    NaN    4.0    NaN    NaN    4.0    NaN    NaN    NaN    NaN   
2          NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
3          NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
4          NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
5          4.0    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   

         ... timestamp                                                   \
movieId  ...    193565 193567 193571 193573 193579 193581 193583 193585   
userId   ...                                                              
1        ...       NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
2        ...       NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
3        ...       NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
4        ...       NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
5        ...       NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   

                       
movieId 193587 193609  
userId                 
1          NaN    NaN  
2          NaN    NaN  
3          NaN    NaN  
4          NaN    NaN  
5          NaN    NaN  

[5 rows x 19448 columns]

In [61]:
rates.rating

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,2.5,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [62]:
# import the data set of movie names
df_movie = pd.read_csv('../data/ml-latest-small/movies.csv', na_values = 'Nan')

In [67]:
# Try to zip columns with movie names
#new_columns = dict(zip(df.movieId,df_movie.title))
rates.rename(columns=dict(zip(df_movie["movieId"], df_movie["title"])),inplace = True)
rates.head()

rating                                         \
movieId Toy Story (1995) Jumanji (1995) Grumpier Old Men (1995)   
userId                                                            
1                    4.0            NaN                     4.0   
2                    NaN            NaN                     NaN   
3                    NaN            NaN                     NaN   
4                    NaN            NaN                     NaN   
5                    4.0            NaN                     NaN   

                                                                     \
movieId Waiting to Exhale (1995) Father of the Bride Part II (1995)   
userId                                                                
1                            NaN                                NaN   
2                            NaN                                NaN   
3                            NaN                                NaN   
4                            NaN                                NaN   
5                            NaN                                NaN   

                                                                            \
movieId Heat (1995) Sabrina (1995) Tom and Huck (1995) Sudden Death (1995)   
userId                                                                       
1               4.0            NaN                 NaN                 NaN   
2               NaN            NaN                 NaN                 NaN   
3               NaN            NaN                 NaN                 NaN   
4               NaN            NaN                 NaN                 NaN   
5               NaN            NaN                 NaN                 NaN   

                          ...                 timestamp  \
movieId GoldenEye (1995)  ... Gintama: The Movie (2010)   
userId                    ...                             
1                    NaN  ...                       NaN   
2                    NaN  ...                       NaN   
3                    NaN  ...                       NaN   
4                    NaN  ...                       NaN   
5                    NaN  ...                       NaN   

                                                                \
movieId anohana: The Flower We Saw That Day - The Movie (2013)   
userId                                                           
1                                                      NaN       
2                                                      NaN       
3                                                      NaN       
4                                                      NaN       
5                                                      NaN       

                                                                     \
movieId Silver Spoon (2014) Love Live! The School Idol Movie (2015)   
userId                                                                
1                       NaN                                     NaN   
2                       NaN                                     NaN   
3                       NaN                                     NaN   
4                       NaN                                     NaN   
5                       NaN                                     NaN   

                                                  \
movieId Jon Stewart Has Left the Building (2015)   
userId                                             
1                                            NaN   
2                                            NaN   
3                                            NaN   
4                                            NaN   
5                                            NaN   

                                                   \
movieId Black Butler: Book of the Atlantic (2017)   
userId                                              
1                                             NaN   
2                                             NaN   
3                                 

In [68]:
rate_movies =  rates.rating

In [69]:
rate_movies.

movieId,Toy Story (1995),Jumanji (1995),Grumpier Old Men (1995),Waiting to Exhale (1995),Father of the Bride Part II (1995),Heat (1995),Sabrina (1995),Tom and Huck (1995),Sudden Death (1995),GoldenEye (1995),...,Gintama: The Movie (2010),anohana: The Flower We Saw That Day - The Movie (2013),Silver Spoon (2014),Love Live! The School Idol Movie (2015),Jon Stewart Has Left the Building (2015),Black Butler: Book of the Atlantic (2017),No Game No Life: Zero (2017),Flint (2017),Bungo Stray Dogs: Dead Apple (2018),Andrew Dice Clay: Dice Rules (1991)
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
